In [2]:
import torch
import torch.nn as nn

In [3]:
random = torch.rand(1,2,3)
print(random) , print(random.shape)
b,s,e = random.size()
b,s,e

tensor([[[0.8341, 0.0153, 0.6267],
         [0.5595, 0.5399, 0.5244]]])
torch.Size([1, 2, 3])


(1, 2, 3)

In [4]:
input = random.reshape(s,b,e)
input.shape

torch.Size([2, 1, 3])

In [5]:
parameter_shape = input.size()[-2:]
parameter_shape

torch.Size([1, 3])

In [6]:
gamma = torch.ones(parameter_shape)
beta = torch.zeros(parameter_shape)
gamma,gamma.shape,beta,beta.shape

(tensor([[1., 1., 1.]]),
 torch.Size([1, 3]),
 tensor([[0., 0., 0.]]),
 torch.Size([1, 3]))

In [7]:
len(parameter_shape)

2

In [8]:
dims = [-(i+1) for i in range(len(parameter_shape))]
dims

[-1, -2]

Varience:

( \sigma^2 ) is the population variance.

( X_i ) represents each value in the population.

( \mu ) is the population mean.

( N ) is the number of values in the population

$$
\text{σ2} = ∑\bigg(\frac{(Xi​−μ)^2}{\N}\bigg)
$$

In [9]:
Pop_mean = input.mean(dim=dims,keepdim=True)

In [10]:
Pop_mean,Pop_mean.shape,input

(tensor([[[0.4920]],
 
         [[0.5413]]]),
 torch.Size([2, 1, 1]),
 tensor([[[0.8341, 0.0153, 0.6267]],
 
         [[0.5595, 0.5399, 0.5244]]]))

In [11]:
a = (0.1732-0.4662)**2 + (0.4606-0.4662)**2 +(0.7647-0.4662)**2

In [12]:
a/3

0.05832753666666668

In [13]:
var = ((input-Pop_mean)**2).mean(dim=dims,keepdim=True)
var

tensor([[[0.1208]],

        [[0.0002]]])

In [14]:
epsilon = 1e-5
std = (var+epsilon).sqrt()
std

tensor([[[0.3476]],

        [[0.0147]]])

In [15]:
y = (input-Pop_mean)/std
y

tensor([[[ 0.9842, -1.3715,  0.3874]],

        [[ 1.2399, -0.0932, -1.1467]]])

In [16]:
out = y*gamma + beta

In [17]:
out

tensor([[[ 0.9842, -1.3715,  0.3874]],

        [[ 1.2399, -0.0932, -1.1467]]])

In [18]:
class Normalization():
    def __init__(self,parameter_shape, epl= 1e-5):
        self.parameter_shape = parameter_shape
        self.epl = epl
        self.gamma = nn.Parameter(torch.ones(parameter_shape))
        self.beta = nn.Parameter(torch.zeros(parameter_shape))

    def forward(self,sen):
        dims = [-(i+1) for i in range(len(parameter_shape))]
        print(len(dims))
        pop_mean = sen.mean(dim=dims,keepdim=True)
        print(pop_mean.shape),print(pop_mean)
        var = ((sen-pop_mean)**2).mean(dim=dims,keepdim=True)
        print(var.shape),print(var)
        std = (var+self.epl).sqrt()
        print(std.shape),print(std)
        y = (sen-pop_mean)/std
        print(y.shape)
        out = y*self.gamma + self.beta

        return out
        


In [19]:
batch_size = 3
sentence_length = 5
embedding_dim = 8 
sen = torch.randn(sentence_length, batch_size, embedding_dim)

print(f"input \n ({sen.size()}) = \n {sen[:2][:][:]}")

input 
 (torch.Size([5, 3, 8])) = 
 tensor([[[ 1.0056,  0.3979,  2.2108, -0.6678, -0.4306,  0.1015, -0.7558,
          -0.0436],
         [ 0.0786, -0.3494,  0.9187,  1.7297,  1.7962, -0.6656,  0.4365,
          -1.1339],
         [ 1.3357,  1.2347,  0.6506,  0.6265, -0.0346,  1.9204,  0.4649,
          -2.3551]],

        [[-1.1411,  0.6284,  1.5744,  0.3169, -0.5679, -2.4647, -0.7900,
          -1.1995],
         [ 0.1288, -0.3861,  0.4700, -0.4184, -0.0500,  0.2301, -0.0606,
           0.9324],
         [-1.0688,  0.8302, -0.5264,  1.4820,  2.0900,  0.3958,  0.3762,
          -0.2949]]])


In [20]:
sen.size()[-2:]

torch.Size([3, 8])

In [21]:
lay = Normalization(sen.size()[-1:])

In [22]:
new = lay.forward(sen)
new.shape

2
torch.Size([5, 1, 1])
tensor([[[ 0.3530]],

        [[ 0.0203]],

        [[ 0.0836]],

        [[-0.4199]],

        [[-0.1264]]])
torch.Size([5, 1, 1])
tensor([[[1.1236]],

        [[0.9698]],

        [[0.9105]],

        [[1.1261]],

        [[1.0276]]])
torch.Size([5, 1, 1])
tensor([[[1.0600]],

        [[0.9848]],

        [[0.9542]],

        [[1.0612]],

        [[1.0137]]])
torch.Size([5, 3, 8])


torch.Size([5, 3, 8])

In [23]:
new

tensor([[[ 6.1571e-01,  4.2390e-02,  1.7526e+00, -9.6301e-01, -7.3925e-01,
          -2.3727e-01, -1.0460e+00, -3.7414e-01],
         [-2.5889e-01, -6.6258e-01,  5.3369e-01,  1.2988e+00,  1.3615e+00,
          -9.6095e-01,  7.8742e-02, -1.4027e+00],
         [ 9.2709e-01,  8.3174e-01,  2.8076e-01,  2.5804e-01, -3.6568e-01,
           1.4786e+00,  1.0560e-01, -2.5547e+00]],

        [[-1.1794e+00,  6.1748e-01,  1.5781e+00,  3.0120e-01, -5.9729e-01,
          -2.5233e+00, -8.2279e-01, -1.2387e+00],
         [ 1.1019e-01, -4.1264e-01,  4.5667e-01, -4.4551e-01, -7.1321e-02,
           2.1305e-01, -8.2142e-02,  9.2621e-01],
         [-1.1059e+00,  8.2245e-01, -5.5510e-01,  1.4843e+00,  2.1017e+00,
           3.8132e-01,  3.6146e-01, -3.2010e-01]],

        [[-4.8278e-02, -5.3454e-01,  1.7954e+00,  1.6040e+00,  2.3403e+00,
          -9.9651e-01,  1.1408e-01, -1.0976e-01],
         [ 6.3714e-02, -2.8646e-01, -1.2872e+00, -1.2476e+00, -6.5542e-01,
           4.5147e-01,  9.4858e-03,  7.1902e-0